# Get List of TF NCBI IDs 
Get the list of IDs that will be considered TF, both from GO terms and from TFCheckpoint.

The GO terms and columns from TFCheckpoint used for each of the TF types is detailed in the cell below. The specific procedure used is explained in their respective sections:  [Get GO terms](#get-go-terms) and [GET TFCheckpoint terms](#get-tfcheckpoint-terms)

In [65]:
__import__('sys').path.append('../scripts/common/'); __import__('notebook_utils').table_of_contents('get_NCBI_TF_IDs.ipynb')

<h3>Table of contents</h3>


[Get List of TF NCBI IDs](#Get-List-of-TF-NCBI-IDs)
- [Setup](#Setup)
- [Get GO terms](#Get-GO-terms)
- [Get TFCheckpoint terms](#Get-TFCheckpoint-terms)
- [Get final TF set](#Get-final-TF-set)
- [Filter out less likely coTFs](#Filter-out-less-likely-coTFs)
  - [Get coTF human orthologs for filtering](#Get-coTF-human-orthologs-for-filtering)
  - [Send all coTFs.](#Send-all-coTFs.)

## Setup

In [3]:
# IMPORTS
import pandas as pd
from IPython.display import display, HTML
import requests

from Bio import Entrez
# *Always* tell NCBI who you are
Entrez.email = "example24@gmail.com"

import sys
sys.path.append('../common')
from notebook_utils import h3, h4, h5, md

In [4]:
# GO TERM & TFCHECKPOINT VARIABLES

# GO terms used:
GO_dbTF = ["GO:0003700"]
GO_GTF =  ["GO:0140223"]
GO_coTF = ["GO:0003712", "GO:0001098", "GO:0002039" , "GO:0008134" , "GO:0042393", "GO:0046332", "GO:0006325", "GO:0140993"]

# Columns from TFCheckpoint used:
TFCheckpoint_cols = {
    'dbTF': ['GO:0003700.Annotations', 'TFclass.present.merged', 'lambert_2018.present', 'Lovering_2021.present'],
    'GTF':  ['GO:0140223.Annotations'],
    'coTF': ['animal_tfdb_Homo_sapiens_cofactors.present', 'animal_tfdb_Mus_musculus_cofactors.present', 'animal_tfdb_Rattus_norvegicus_cofactors.present', 
             'tcof_cotf_human.present', 'tcof_cotf_mouse.present']
}

In [5]:
# PATHS
in_data_path = '../../data/external/TF_id/'
postprocessing_path = '../../data/postprocessing/'

less_likely_coTFs_path = postprocessing_path + 'all_coTFs_likely_checked_updated_AL.txt'

QuickGO_dbTF_path = in_data_path + "QuickGO-annotations-dbTF.tsv"
QuickGO_GTF_path  = in_data_path + "QuickGO-annotations-GTF.tsv"
QuickGO_coTF_path = in_data_path + "QuickGO-annotations-coTF.tsv"
TFCheckpoint_path = in_data_path + "TFCheckpoint.tsv"

# Define a function to construct the TF types path (ll_coTFs are introduced later)
TF_types = ["dbTF", "GTF", "coTF"]
def get_TF_ids_path(TF_type, out_data_path):
    return f"{out_data_path}{TF_type}_entrez_code.list"

In [6]:
# LOAD LIKELY & LESS LIKELY COTFs
ll_coTFs_db = pd.read_csv(less_likely_coTFs_path, sep="\t", dtype='str')
m = ll_coTFs_db['likely'] == 'likely'
ll_coTF = set(ll_coTFs_db[~m]['NCBI ID'])

## Get GO terms

Obtained the GO terms from [QuickGO](https://www.ebi.ac.uk/QuickGO/annotations?taxonId=10116,9606,10090&taxonUsage=exact&goId=GO:0140993,GO:0003712,GO:0003700,GO:0140223,GO:0001098,GO:0002039,GO:0008134,GO:0042393,GO:0046332,GO:0006325&goUsageRelationships=is_a,part_of,occurs_in&goUsage=descendants&geneProductSubset=Swiss-Prot&geneProductType=protein), using the terms shown below. Used as filters:

* **Taxon:** 10116, 9606, 10090, Exact match (do not include descendants)
* **Gene products:** Reviewed (not Unreviewed)
* **GO terms:** the ones shown below.
* **Export as:** tsv

Downloaded separately a QuickGO tsv file for each TF type and renamed it as shown above in the setup section.

As some terms can be identified as pertaining to more than 1 type, we have followed this hierarchy to remove duplicates:
1. dbTF
2. GTF
3. coTF

That implies that if a protein is classified as both dbTF and GTF, the protein's classification will be dbTF.

In [10]:
# VARIABLES
# Species:
organismToTaxID = {
    "hsapiens": "9606",
    "mmusculus": "10090",
    "rnorvegicus": "10116"}

In [11]:
# FUNCTIONS
def fetch_gene_ids_gprofiler(gene_symbols: list, organism: str) -> dict:
    "Get NCBI Gene IDs from GProfiler"
    symboltoID = {}

    # Query the IDs from GProfiler
    result = requests.post(
        url='https://biit.cs.ut.ee/gprofiler/api/convert/convert/',
        json={
            'organism': organism,
            'target':'ENTREZGENE_ACC',
            'query': gene_symbols,
        }
        )
    
    # Create a list of extracted IDs per symbol
    for r in result.json()['result']:
        incoming = r['incoming']
        converted = r['converted']

        if incoming not in symboltoID:
            symboltoID[incoming] = []
        if converted != 'None':
            symboltoID[incoming].append(converted)

    return symboltoID

def retrieve_annotations_entrez(id_list):
    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene", id=",".join(id_list))
    result = Entrez.read(request)
    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
    annotations = Entrez.read(data)
    annotationsSummary = annotations['DocumentSummarySet']['DocumentSummary']

    assert len(id_list) == len(annotationsSummary), f"id_list and annotationsSummary are of different length: {len(id_list)} != {len(annotationsSummary)}"

    return annotationsSummary

In [12]:
# JOIN QUICKGO TSVs
# Create joined DataFrame from the 3 TF types
QuickGO_dbTF = pd.read_csv(QuickGO_dbTF_path, sep='\t', header=0, keep_default_na=False, dtype='str')
QuickGO_dbTF['TF type'] = 'dbTF' 
QuickGO_GTF  = pd.read_csv(QuickGO_GTF_path,  sep='\t', header=0, keep_default_na=False, dtype='str')
QuickGO_GTF['TF type'] = 'GTF' 
QuickGO_coTF = pd.read_csv(QuickGO_coTF_path, sep='\t', header=0, keep_default_na=False, dtype='str')
QuickGO_coTF['TF type'] = 'coTF' 
QuickGO = pd.concat([QuickGO_dbTF, QuickGO_GTF, QuickGO_coTF], axis=0)

print(f"{len(QuickGO['SYMBOL'])} rows were retrieved.")

# Only keep relevant columns
QuickGO = QuickGO[['SYMBOL', 'TAXON ID', 'TF type']]

# Drop repeated cells. Use the following priority if duplicates of different TF type
priority = {'dbTF': 0, 'GTF': 1, 'coTF': 2}
QuickGO['priority'] = QuickGO['TF type'].map(priority)
QuickGO = QuickGO.sort_values(by=['SYMBOL', 'TAXON ID', 'priority'])
QuickGO = QuickGO.drop_duplicates(subset=['SYMBOL', 'TAXON ID'], keep='first')
QuickGO.drop(columns=['priority'], inplace=True) 

# Show results
print(f"Removing duplicates, we retrieve {len(QuickGO['SYMBOL'])} symbols:")
for TF_type in ('dbTF', 'GTF', 'coTF'):
    print(f"\t{len(QuickGO[QuickGO['TF type'] == TF_type]['SYMBOL'])} {TF_type}s")

25259 rows were retrieved.
Removing duplicates, we retrieve 6076 symbols:
	2849 dbTFs
	120 GTFs
	3107 coTFs


In [13]:
# GET GENE IDs FROM GPROFILER
for organism in ['hsapiens', 'mmusculus', 'rnorvegicus']:
    # Get IDs from GProfiler
    symbols = list(QuickGO[QuickGO['TAXON ID'] == organismToTaxID[organism]]['SYMBOL'].unique())
    symboltoID = fetch_gene_ids_gprofiler(symbols, organism)

    # Map them to QuickGO db
    m = QuickGO['TAXON ID'] == organismToTaxID[organism]
    QuickGO.loc[m, "TF ID"] = QuickGO[m]['SYMBOL'].apply(lambda symbol: symboltoID[symbol])

m = ~(QuickGO['TF ID'].str.len() == 0)
print(f'GProfiler retrieved {m.sum() / len(QuickGO):.1%} NCBI Gene IDs from the QuickGO symbols')
print(f"It couldn't retrieve {(~m).sum()} of them")

GProfiler retrieved 98.4% NCBI Gene IDs from the QuickGO symbols
It couldn't retrieve 99 of them


We retrieve the rest through a query to Entrez.

In [15]:
# QUERY THE REST FROM ENTREZ
# Entrez often gets stuck, so it's best to give some time between queries
import time
ids = []
for TaxID in ['9606', '10090', '10116']:

    # Get the symbols with missing ID
    m = (QuickGO['TF ID'].str.len() == 0) & (QuickGO['TAXON ID'] == TaxID)
    missing_symbols = list(QuickGO[m]['SYMBOL'].unique())

    nSymbols = 15 # Symbols per query
    for i in range(0, len(missing_symbols), nSymbols):
        symbols = missing_symbols[i:i+nSymbols]
        # Query them from Entrez
        symbolsQuery = sorted([s+'[Preferred Symbol]' for s in symbols])
        query = f'({" OR ".join(symbolsQuery)}) AND txid{TaxID}[Organism]'
        handle = Entrez.esearch(db="gene", term=query, retmode="xml")
        record = Entrez.read(handle)
        ids.append(record.get("IdList", []))
        print(TaxID)
        # Sleep between queries to not get blocked
        time.sleep(5)

KeyboardInterrupt: 

In [37]:
# Map IDs back to its symbol & organism
all_ids = [j for i in ids for j in i]
remaining_annotations = retrieve_annotations_entrez(all_ids)

# Make a map from symbol/TaxID to Gene ID
symboltoID_entrez = {'9606': {}, '10116': {}, '10090': {}}
for id, ann in zip(all_ids, remaining_annotations):
    symbol = ann['Name']
    TaxID = ann['Organism']['TaxID']

    if symbol not in symboltoID_entrez[TaxID]:
        symboltoID_entrez[TaxID][symbol] = [id]
    else:
        symboltoID_entrez[TaxID][symbol].append(id)

# Check how many we retrieved from Entrez
m = (QuickGO['TF ID'].str.len() == 0)
print(f"{len(all_ids)} out of the remaining {m.sum()} missing have been retrieved through Entrez.")

# Map the retrieved ones to the QuickGO db
for TaxID in symboltoID_entrez.keys():
    m = (QuickGO['TF ID'].str.len() == 0) & (QuickGO['TAXON ID'] == TaxID)
    QuickGO.loc[m, "TF ID"] = QuickGO[m]['SYMBOL'].apply(lambda symbol: symboltoID_entrez[TaxID].get(symbol, []))

m = ~(QuickGO['TF ID'].str.len() == 0)
print(f'Combined with Entrez, we retrieved {m.sum() / len(QuickGO):.1%} NCBI Gene IDs from the QuickGO symbols')
print(f"There's {(~m).sum()} NCBI Gene IDs that couldn't be retrieved")

63 out of the remaining 99 missing have been retrieved through Entrez.
Combined with Entrez, we retrieved 99.3% NCBI Gene IDs from the QuickGO symbols
There's 40 NCBI Gene IDs that couldn't be retrieved


## Get TFCheckpoint terms
Cells copied from `analysis.ipynb`

In [10]:
## LOAD & PREPROCESS TFCHECKPOINT TSV
# Load TFCheckpoint dataset
TF_df = pd.read_csv(TFCheckpoint_path, sep='\t', header=0)
str_cols = ['Associated.Gene.Name', 'Synonyms', 'Official name', 'Entrez.Taxa.ID', 'Entrez.Gene.ID', 'UniProt.SwissProt.Accession', 'Ensembl.Gene.ID']
TF_df[str_cols] = TF_df[str_cols].astype(str)


# Split Entrez, Taxa & UniProt into individual IDs
TF_df['EntrezIDs'] = TF_df['Entrez.Gene.ID'].str.split('|')
TF_df['TaxaIDs'] = TF_df['Entrez.Taxa.ID'].str.split('|')
TF_df['UniProt'] = TF_df['UniProt.SwissProt.Accession'].str.split('|')
TF_df['Ensembl'] = TF_df['Ensembl.Gene.ID'].str.split('|')

# Explode the TF
TF_exploded = TF_df.explode(['EntrezIDs', 'TaxaIDs', 'UniProt', 'Ensembl'])
TF_exploded = TF_exploded[TF_exploded["EntrezIDs"] != ''] # Drop empty rows (Appeared when | was present at the end, e.g. "9454|3425|")
TF_exploded = TF_exploded[TF_exploded["UniProt"] != '']
TF_exploded = TF_exploded[TF_exploded["Ensembl"] != '']

# Check whether each EntrezID only matches to 1 TaxaID:
gene_taxa_unique = TF_exploded.drop_duplicates(subset=["EntrezIDs", "TaxaIDs"], keep='first')
gene_taxa_mismatch = gene_taxa_unique[gene_taxa_unique.duplicated(subset=["EntrezIDs"], keep=False)]
h4("EntrezIDs mapped to 2 species")
md(f"There are {len(gene_taxa_mismatch['EntrezIDs'].unique())} Entrez IDs that are mapped to both Rat and Mouse:")
display(HTML(gene_taxa_mismatch[["EntrezIDs", "Associated.Gene.Name", "TaxaIDs"]].sort_values(by=['EntrezIDs']).to_html(index=False)))

# DROP MISMATCHING ROWS
rows_to_drop = [
    ['STAT5A', '20851', '10116'],
    ['STAT5A', '25126', '10090'],
    ['ZFY', '367832', '10090'],
    ['ZFY', '22764', '10116'],
    ['STAT5B', '24918', '10116']
]
for row in rows_to_drop:
    to_drop = (TF_exploded["Associated.Gene.Name"] == row[0]) & (TF_exploded["EntrezIDs"] == row[1])
    assert to_drop.sum() == 1, f"{to_drop.sum()} rows are being dropped instead of 1"
    TF_exploded = TF_exploded[~to_drop]
to_change = (TF_exploded["Associated.Gene.Name"] == "STAT5A") & (TF_exploded["EntrezIDs"] == "24918")
assert to_change.sum() == 1, f"{to_change.sum()} rows are being dropped instead of 1"
TF_exploded.loc[to_change, "TaxaIDs"] = "10116"
md("They have been searched in the NCBI and corrected manually")

# Assert there's no duplicates anymore
gene_taxa_unique = TF_exploded.drop_duplicates(subset=["EntrezIDs", "TaxaIDs"], keep='first')
gene_taxa_mismatch = gene_taxa_unique[gene_taxa_unique.duplicated(subset=["EntrezIDs"], keep=False)]
assert len(gene_taxa_mismatch) == 0, f"There's still {len(gene_taxa_mismatch)} duplicated rows" 

<h4>EntrezIDs mapped to 2 species</h4>

There are 5 Entrez IDs that are mapped to both Rat and Mouse:

EntrezIDs,Associated.Gene.Name,TaxaIDs
20851,STAT5A,10116
20851,STAT5B,10090
22764,ZFX,10090
22764,ZFY,10116
24918,STAT5A,10090
24918,STAT5B,10116
25126,STAT5A,10090
25126,STAT5B,10116
367832,ZFX,10116
367832,ZFY,10090


They have been searched in the NCBI and corrected manually

In [11]:
# GROUP DUPLICATED ROWS & GET FINAL TFCHECKPOINT DATASET
# In some rows, EntrezID, TaxaID & Name are the same -> Only SwissProt changes. We will group those rows

# Remove all useless columns
columns_to_keep = TF_exploded.columns.tolist()
columns_to_remove = ['Entrez.Taxa.ID', 'Entrez.Gene.ID', 'UniProt.SwissProt.Accession', 'Ensembl.Gene.ID', 'UniProt', 'Ensembl']
for column in columns_to_remove:
    columns_to_keep.remove(column)

# Group duplicated rows, with a | in between for UniProt & Ensembl.
TF_exploded = TF_exploded.groupby(columns_to_keep, dropna=False).agg({
    "UniProt": lambda x: "|".join(x),
    "Ensembl": lambda x: "|".join(x)
}).reset_index()

# Display one example
mask = TF_exploded["UniProt"].str.contains("\|")
md(f"In {mask.sum()} TFs, one EntrezID is mapped to 2 different SwissProt Accession IDs. They have been joined by |. Example:")
display(HTML(TF_exploded[mask][:2][["Associated.Gene.Name", "Official name", "EntrezIDs", "TaxaIDs", "UniProt", "Ensembl"]].to_html(index=False)))

In 16 TFs, one EntrezID is mapped to 2 different SwissProt Accession IDs. They have been joined by |. Example:

Associated.Gene.Name,Official name,EntrezIDs,TaxaIDs,UniProt,Ensembl
ABL1,Tyrosine-protein kinase ABL1,100909750,10116,E9PT20|F1M0A6,ENSRNOG00000009371|ENSRNOG00000009371
CHCHD2,Coiled-coil-helix-coiled-coil-helix domain-containing protein 2,316643,10116,Q5BJB3|M0R785,ENSRNOG00000051180|ENSRNOG00000051180


In [27]:
TFCheckpoint_sets = {}
for TF_type in TF_types:
    mask = TF_exploded[TFCheckpoint_cols[TF_type]].notna().any(axis=1)  # Checks across the specified columns
    TFCheckpoint_sets[TF_type] = set(TF_exploded[mask]['EntrezIDs'])
    print(f"# {TF_type} NCBI IDs in TFCheckpoint: {len(TFCheckpoint_sets[TF_type]):>5}")

# dbTF NCBI IDs in TFCheckpoint:  4460
# GTF NCBI IDs in TFCheckpoint:   146
# coTF NCBI IDs in TFCheckpoint:  3598


## Get final TF set

In [49]:
TF_IDs_dict = {}
for TF_type in TF_types:
    TF_IDs = set()

    # Get QuickGO TFs
    QuickGO_subset = QuickGO[QuickGO['TF type'] == TF_type]
    QuickGO_IDs = [j for i in list(QuickGO_subset['TF ID']) for j in i]
    TF_IDs.update(set(QuickGO_IDs))
        
    # Get TFCheckpoint TFs
    TF_IDs.update(TFCheckpoint_sets[TF_type])

    # Save into dictionary
    TF_IDs_dict[TF_type] = TF_IDs


# GTFs must not contain dbTFs, and coTFs must not contain neither dbTFs nor GTFs.
TF_IDs_dict['GTF'].difference_update(TF_IDs_dict['dbTF'])
TF_IDs_dict['coTF'].difference_update(TF_IDs_dict['dbTF'])
TF_IDs_dict['coTF'].difference_update(TF_IDs_dict['GTF'])

# Add the less likely coTFs as a subset of the coTFs
TF_IDs_dict['ll_coTF'] = ll_coTF.intersection(TF_IDs_dict['coTF'])

# Save them as a list
for TF_type in TF_types + ['ll_coTF']:
    path = get_TF_ids_path(TF_type, postprocessing_path)
    with open(path, 'w') as f:
        for TF in TF_IDs_dict[TF_type]:
            f.write(TF + "\n")


# Combine all TFs & save them as a list
all_TF_ids = TF_IDs_dict['dbTF'].union(TF_IDs_dict['GTF']).union(TF_IDs_dict['coTF'])
print(f"We consider {len(all_TF_ids)} NCBI Gene IDs to be TFs")

with open(get_TF_ids_path('tf', postprocessing_path), 'w') as f:
    for TF in all_TF_ids:
        f.write(TF + "\n")

We consider 9297 NCBI Gene IDs to be TFs


## Filter out less likely coTFs

### Get coTF human orthologs for filtering
coTFs are converted to their human orthologs to then apply a filtering and discard non-coTFs.

The previous list of coTFs is polluted with proteins that do not act as coTFs. They are filtered out by analysing their human orthologs. For that:
1. coTFs are separarted in human & mouse/rat coTFs.
2. Human orthologs for mouse/rat coTFs are oubtained. Those mouse/rat coTFs without human orthologs (24/2822 cases) are discarded.
3. Mouse/rat IDs frequently have +1 human orthologs. To not pollute the final coTF list with unnecessary orthologs, we check, for each mouse/rat coTF, if any of the human orthologs is already in the human coTF list. The majority has at least one, so the other orthologs were not added.
4. For those that did not have any ortholog in the human coTF list (99 cases), we add all their human orthologs (115 additional ones).
5. We create a TSV file with the resulting 1876 human coTF IDs + their NCBI Symbol, which is used to find filtering strategies.

In [22]:
# PATHS
data_path = '../../data/'
orthologs_path =       data_path + 'external/human_HGNC_orthologs/'
all_human_coTFs_path = data_path + 'tmp/all_human_ortholog_coTFs.tsv'
r_m_coTFs_wo_orthologs_path = data_path + 'tmp/r_m_coTFs_wo_orthologs.tsv'

In [23]:
# FUNCTIONS
from Bio import Entrez
import os
Entrez.email = "example24@gmail.com"

def retrieve_annotations(id_list):
    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene", id=",".join(id_list))
    result = Entrez.read(request)
    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
    annotations = Entrez.read(data)
    annotationsSummary = annotations['DocumentSummarySet']['DocumentSummary']

    assert len(id_list) == len(annotationsSummary), f"id_list and annotationsSummary are of different length: {len(id_list)} != {len(annotationsSummary)}"

    return annotationsSummary

def get_HGNC_orthologs(coTF_ids: list, orthologs_path: str):
    orthologs = pd.DataFrame()

    for rodent in ['mouse', 'rat']:
        # Load dataframe
        path = os.path.join(orthologs_path, f"human_{rodent}_hcop_fifteen_column.txt")
        hgnc_df = pd.read_csv(path, sep='\t', header=0, keep_default_na=False, dtype=str)

        # Rename
        hgnc_df = hgnc_df.rename(columns={f"{rodent}_entrez_gene": "entrez_gene", f"{rodent}_symbol": "symbol"})
        
        orthologs = pd.concat([orthologs, hgnc_df])

    # Keep only IDs present in the coTF list
    orthologs = orthologs[orthologs['entrez_gene'].isin(coTF_ids)]

    # Remove all rows that don't have a human entrez ID or hgnc ID
    m = (orthologs['human_entrez_gene'] != '-') | (orthologs['hgnc_id'] != '-')
    orthologs = orthologs[m]

    # Join with ';' when an EntrezID has more than 1 human ortholog
    agg_funcs = {
        "symbol": lambda x: ';'.join(x.unique()),
        "human_entrez_gene": lambda x: ';'.join(x.unique()),
        "hgnc_id": lambda x: ';'.join(x.unique()),
        "human_symbol": lambda x: ';'.join(x.unique())
    }
    orthologs = orthologs.groupby(['entrez_gene']).agg(agg_funcs).reset_index()

    # Show how many we get
    print(f"We get ortholog info for {len(orthologs)}/{len(coTF_ids)} Gene IDs")


    # Convert into dictionary
    orthologs_map = orthologs.set_index('entrez_gene').to_dict(orient='index')

    return orthologs_map

In [24]:
# Load coTF IDs list
path = get_TF_ids_path('coTF', out_data_path+"postprocessing/")
with open(path, 'r') as f:
    coTF_ids = [l.strip('\n') for l in f]

# Create map between the ID & its Name / TaxID
annotationsSummary = retrieve_annotations(coTF_ids)
EntrezIDtoSymbol = {ID : {'Name': annotation["Name"], 'TaxID': annotation['Organism']['TaxID']} for ID, annotation in zip(coTF_ids, annotationsSummary)}

# Separate coTFs between human & mouse/rat
human_coTFs = {id for id, val in EntrezIDtoSymbol.items() if val['TaxID'] == '9606'}
m_r_coTFs   = {id for id, val in EntrezIDtoSymbol.items() if val['TaxID'] != '9606'}

# Get mouse/rat human orthologs as a dataframe
orthologs_map = get_HGNC_orthologs(m_r_coTFs, orthologs_path)
orth_df = pd.DataFrame.from_dict(orthologs_map, orient='index')
orth_df.index.name = 'ID'
r_m_coTFs_wo_orthologs = m_r_coTFs.difference(set(orth_df.index))

# Mask for mouse/rat IDs with an ortholog inside the human_coTFs list
def contains_coTF(ids: str, coTFs: set):
    return any(id in coTFs for id in ids.split(';'))
m = orth_df['human_entrez_gene'].apply(lambda ids: contains_coTF(ids, human_coTFs))
print(f"In {m.sum()} / {len(orth_df)} mouse/rat IDs, at least one of their orthologs is also in the human_coTFs list.")

# Get all human orthologs of mouse/rat IDs w/o orthologs already present in human_coTFs
remaining_m_r_human_orthologs = {id for id_list in orth_df[~m]['human_entrez_gene'] for id in id_list.split(';')}
print(f"For those that don't, we'll take all its human orthologs: {len(remaining_m_r_human_orthologs)} extra human IDs")

# Join human coTFs + the other human orthologs.
all_human_IDs = remaining_m_r_human_orthologs | human_coTFs

# Get the symbol from Entrez & save the list as a tsv file.
annotationsSummary = retrieve_annotations(all_human_IDs)
EntrezIDtoSymbol = {ID : {'Name': annotation["Name"], 'TaxID': annotation['Organism']['TaxID']} for ID, annotation in zip(coTF_ids, annotationsSummary)}
all_human_coTFs_df = pd.DataFrame.from_dict(EntrezIDtoSymbol, orient='index')
all_human_coTFs_df.index.name = 'ID'
all_human_coTFs_df.to_csv(all_human_coTFs_path, sep='\t')

# Do the same for the mouse/rat coTFs without human orthologs
annotationsSummary = retrieve_annotations(r_m_coTFs_wo_orthologs)
EntrezIDtoSymbol = {ID : {'Name': annotation["Name"], 'TaxID': annotation['Organism']['TaxID']} for ID, annotation in zip(r_m_coTFs_wo_orthologs, annotationsSummary)}
r_m_coTFs_wo_orthologs_df = pd.DataFrame.from_dict(EntrezIDtoSymbol, orient='index')
r_m_coTFs_wo_orthologs_df.index.name = 'ID'
r_m_coTFs_wo_orthologs_df.to_csv(r_m_coTFs_wo_orthologs_path, sep='\t')

We get ortholog info for 2798/2822 Gene IDs
In 2699 / 2798 mouse/rat IDs, at least one of their orthologs is also in the human_coTFs list.
For those that don't, we'll take all its human orthologs: 115 extra human IDs


In [25]:
# Do the same for the mouse/rat coTFs without human orthologs
annotationsSummary = retrieve_annotations(r_m_coTFs_wo_orthologs)
EntrezIDtoSymbol = {ID : {'Name': annotation["Name"], 'TaxID': annotation['Organism']['TaxID']} for ID, annotation in zip(r_m_coTFs_wo_orthologs, annotationsSummary)}
r_m_coTFs_wo_orthologs_df = pd.DataFrame.from_dict(EntrezIDtoSymbol, orient='index')
r_m_coTFs_wo_orthologs_df.index.name = 'ID'
r_m_coTFs_wo_orthologs_df.to_csv(r_m_coTFs_wo_orthologs_path, sep='\t')

### Send all coTFs.

This method did not properly separate them: there were still several "less likely" coTFs in the "likely" list. Therefore, we use a different approach:
1. Get all coTFs. Sort them alphabetically (case insensitive). 
2. Create 'likely' column, and mark it as "less likely" if the symbol is present in the list of less_likely_human_coTFs (case insensitive)
3. Create an Excel, to recheck the likely columns

In [63]:
# Get all coTF Symbols from Entrez
annotationsSummary = retrieve_annotations(coTF_ids)
EntrezIDtoSymbol = {ID : {'Name': annotation["Name"], 'TaxID': annotation['Organism']['TaxID']} for ID, annotation in zip(coTF_ids, annotationsSummary)}
all_coTFs = pd.DataFrame.from_dict(EntrezIDtoSymbol, orient='index')
all_coTFs.index.name = 'ID'
all_coTFs.sort_values(by='Name', inplace=True, key=lambda col: col.str.lower())

# Get list of less likely human coTFs
data_path = '../../data/'
orthologs_path               = data_path + 'external/human_HGNC_orthologs/'
ll_human_coTFs_path = data_path + 'postprocessing/all_human_ortholog_coTFs_labelled_less_likely.txt'
all_coTFs_likely_path =        data_path + 'tmp/all_coTFs_likely.tsv'

# Get the set of the human IDs less likely to be coTFs
human_coTFs_labelled = pd.read_csv(ll_human_coTFs_path, sep='\t', header=0, dtype='str')
h_ll_coTFs_symbol = set(human_coTFs_labelled[human_coTFs_labelled['less likely'] == 'less likely']['Name'])

# Create a "likely" column
all_coTFs['likely'] = all_coTFs['Name'].str.upper().isin(h_ll_coTFs_symbol).apply(lambda x: 'less likely' if x else 'likely')

# Rename columns
all_coTFs = all_coTFs.rename(columns={'Name': 'NCBI Symbol', 'ID': 'NCBI ID'})
all_coTFs.index.name = 'NCBI ID'

# Save
all_coTFs.to_csv(all_coTFs_likely_path, sep='\t')